## Make memory

In [2]:
from typing import Annotated
from typing_extensions import TypedDict

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

class State(TypedDict):
    messages: Annotated[list, add_messages]

tool = TavilySearchResults(max_results=2)
tools = [tool]
tool_node = ToolNode(tools)

llm = ChatOpenAI(model='gpt-4o-mini')
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    result = llm_with_tools.invoke(state['messages'])
    return {'messages': [result]}

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")

graph = graph_builder.compile(checkpointer=memory)

In [3]:
config = {"configurable": {"thread_id": "1"}}

while True:
    user_input = input("User: ")
    if user_input.lower() in ['quit', 'exit', 'q']:
        print("Goodbye!")
        break
    for event in graph.stream({"messages": ("user", user_input)}, config):
        for value in event.values():
            print("Assistant: ", value["messages"][-1].content)

User:  현재 대한민국의 대통령은 누구야?


Assistant:  
Assistant:  [{"url": "https://namu.wiki/w/대한민국+대통령", "content": "대한민국 대통령은 대한민국의 대통령 기관(機關)·직위(職位) 및 이를 수행하는 인물을 가리킨다. 대한민국의 현직 대통령은 제20대 윤석열이며, 임기는 2022년 5월 10일부터 2027년 5월 9일까지이다. 대통령은 대한민국 헌법 제66조에서 정의하는 헌법기관으로, 국가원수 [원]로서 외국에 대하여 국가를"}, {"url": "https://ko.wikipedia.org/wiki/대한민국의_대통령_목록", "content": "개인 도구 대한민국의 대통령 목록 도구 위키백과, 우리 모두의 백과사전. 제9차 개헌 이후의 현행 헌법에 따라 대통령의 임기는 5년이며 중임할 수 없지만,[1] 제9차 개헌 이전의 여러 헌법에서는 대통령의 임기가 고정되지 않았으며, 그로 인해 대통령직을 수행한 개인과 대통령 임기의 대수가 불일치하게 되었다. (생몰년)   정당  대[b]    임기  선거 / 득표율[5] ↑ 가 나 2016년 헌법에 위배되는 박근혜-최순실 게이트, 비선실세 의혹, 대기업 뇌물 의혹 등을 사유로 박근혜에 대한 탄핵소추안이 국회에서 가결되었고, 대통령 권한이 정지되어 황교안 국무총리가 권한대행을 맡았다.[27] 결과적으로 2017년 3월 10일 박근혜는 헌법재판소에 의해 전원일치로 대통령직에서 파면되었다.[28][29] ↑ 허정 (1979). ↑ KBS 뉴스 (2004). ↑ KBS 뉴스 (2017). ↑ 중앙일보 후보자 정보 (윤석열). 14 김영삼15 김대중16 노무현17 이명박18 박근혜19 문재인20 윤석열 대한민국의 대통령과 3부요인 및 헌법기관장 분류: 알찬 목록 후보 문서대한민국의 대통령대한민국 정치에 관한 목록대통령 목록대한민국 사람 목록 위키백과 소개"}]
Assistant:  현재 대한민국의 대통령은 윤석열입니다. 그는 2022년 5월 10일에 취임하였으며, 임기는 2027년 5월 9일까지입니다.


User:  내가 한 첫질문이 뭔지 기억해?


Assistant:  네, 당신의 첫 질문은 "현재 대한민국의 대통령은 누구야?"였습니다.


User:  q


Goodbye!


In [4]:
snapshot = graph.get_state(config)
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='현재 대한민국의 대통령은 누구야?', additional_kwargs={}, response_metadata={}, id='badeed0c-0019-418f-89ac-2a2b0f6bc8e2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lPlMrBMYhgazADPNCPA9ugZJ', 'function': {'arguments': '{"query":"현재 대한민국 대통령"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 87, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b6e64093-1d23-4e8c-a967-afb5ea03f231-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '현재 대한민국 대통령'}, 'id': 'call_lPlMrBMYhgazADPNCPA9ugZ

### Limit the number of messages to remember

In [5]:
def filter_messages(messages: list):
    return messages[-2:]

def chatbot(state: State):
    messages = filter_messages(state["messages"])
    result = llm_with_tools.invoke(messages)
    return {"messages": [result]}

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")

graph = graph_builder.compile(checkpointer=memory)

In [6]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "15"}}

input_message = HumanMessage(content="Hi~ I'm noahyun I like game")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

input_message = HumanMessage(content="What's my name??")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

input_message = HumanMessage(content="What's my name??")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

input_message = HumanMessage(content="What's my favorite??")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi~ I'm noahyun I like game
================================== Ai Message ==================================

Hi Noahyun! It's great to meet you! What kind of games do you enjoy playing?
================================ Human Message =================================

What's my name??
================================== Ai Message ==================================

Your name is Noahyun!
================================ Human Message =================================

What's my name??
================================== Ai Message ==================================

Your name is Noahyun!
================================ Human Message =================================

What's my favorite??
================================== Ai Message ==================================

I don't have access to personal information about you, including your favorites. However, if you share what you're interested in, I can help